In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Load EMC reconstruction

In [ ]:
emc_folder = path_to_recons + "melv_0001/"
emc_output = emc_folder + "data/r1/output_060.h5"
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_lowq8.h5"

In [ ]:
photons_list = [emc_folder + pl for pl in np.loadtxt(emc_photons, dtype=str)]

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]
    scale = f['scale'][:]

In [ ]:
nr_rot = 180
nr_static = 5
nr_modes = occupancies.shape[1] - nr_static
modes = orientations % (nr_modes)
rotind = orientations // (nr_modes)
modes[rotind >= nr_rot] = orientations[rotind >= nr_rot] - nr_modes * (nr_rot - 1)
blacklisted = (occupancies == 0).all(axis=1)
ndata = len(modes) - blacklisted.sum()

In [ ]:
det = detector.Detector(emc_detector, mask_flag=True)
emc = reademc.EMCReader(photons_list, det)

## Select most interesting 2D class

In [ ]:
sclass = 19
select = np.where(modes==sclass)[0]
scalesort = np.sort(scale[select])[::-1]

## Show 2D class image

In [ ]:
import spimage
rmask = spimage.rmask(images[sclass].shape, 180)

In [ ]:
# customized colors
cmap = cm.get_cmap('inferno')
cmap.set_bad('k')
cmap.set_under('k')

In [ ]:
c = 90
img = images[sclass]
img[img<1e-4] = 1e-4
plt.figure(figsize=(10,10), dpi=200)
plt.axis('off')
plt.imshow((img*rmask)[c:-c,c:-c], norm=colors.LogNorm(vmin=1e-3, vmax=5), cmap='inferno')
plt.show()

## Strongest individual shots

In [ ]:
k = 24
j = np.where(scale==scalesort[k])[0][0]
fr = emc.get_frame(j).astype(np.float)
fr[fr==0] = 1e-2

In [ ]:
# customized colors
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.25')
cmap.set_under('k')

In [ ]:
c = 90
plt.figure(figsize=(10,10), dpi=200)
plt.axis('off')
plt.imshow((fr)[c:-c,c:-c], norm=colors.LogNorm(vmin=1, vmax=None), cmap='inferno')
plt.show()

## Overview of strong shots

In [ ]:
# customized colors
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.25')
cmap.set_under('k')

In [ ]:
N = 25
frames = [emc.get_frame(np.where(scale==scalesort[k])[0][0]).astype(np.float) for k in range(5,N+5)]
for f in frames:
    f[f==0] = 1e-2

In [ ]:
cr = 90
ncols = 5
nrows = N//ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols,nrows), dpi=200)
for i in range(N):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
    #axes[c,r].set_title("%d" %(i))
    axes[c,r].imshow(frames[i][cr:-cr,cr:-cr], norm=colors.LogNorm(vmin=1, vmax=None), cmap='inferno')
    #axes[c,r].text(0,0, "%d" %i, color='r')